# xtuples

[![PyPI - Version](https://img.shields.io/pypi/v/xtuples.svg)](https://pypi.org/project/xtuples)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/xtuples.svg)](https://pypi.org/project/xtuples)


-----

**Table of Contents**

- [Installation](#installation)
- [License](#license)



## Installation

```console
pip install xtuples
```

## Docs Imports

In [1]:
from src import xtuples
import importlib

In [2]:
import typing
import collections
import functools
import itertools
import operator
import timeit
from pympler.asizeof import asizeof

## xtuples

xtuples is designed to make functional programming easier in Python.

The two key constructs are:

- xtuples.iTuple: a base class for iterable data types, equipped with methods like .map(), .fold(), .pipe() and .partial().

- xuples.nTuple.decorate: a decorator to inject .pipe() and .partial() into user-defined NamedTuples (as we can't subclass them directly).

As briefly demonstrated above, by equipping our data structures with .pipe() and .partial(), we're able to use method chaining to mimic the functional pipelines seen in languages like f#.

This then tends to lead us away from inheritance, and more towards composition: to a code base comprised entirely of either free functions, or descendants of either of the above.

### Performance

Performance should generally be at worst, not (materially) worse than a non-optimised canonical equivalent, and in some cases is actually slightly better.

#### iTuple

For instance, iTuple is only a minimal wrapper around the built in tuple.

As such, creation takes very similar time to that of a raw list (a raw tuple would possibly be faster):

In [5]:
%timeit xtuples.iTuple.range(1000)
%timeit list(range(1000))

10.7 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.3 µs ± 493 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Memory usage is very similar:

In [6]:
memory = {}
for i in range(5):
    memory[i] = dict(
        iTuple=asizeof(xtuples.iTuple.range(10 ** i)),
        list=asizeof(list(range(10 ** i))),
    )
memory

{0: {'iTuple': 448, 'list': 88},
 1: {'iTuple': 808, 'list': 448},
 2: {'iTuple': 4408, 'list': 4048},
 3: {'iTuple': 40408, 'list': 40048},
 4: {'iTuple': 400408, 'list': 400048}}

In [7]:
ex_iTuple = xtuples.iTuple.range(1000)
ex_list = list(range(1000))

Iteration is very similar:

In [8]:
%timeit for x in ex_iTuple: pass
%timeit for x in ex_list: pass

8.68 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.2 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
8.19 µs ± 498 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
8.26 µs ± 448 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Though indexing with the .get() method is a little slower (and standard square bracket indexing even slower still - bordering maybe on a gotcha):

In [9]:
%timeit for i in range(1000): ex_iTuple.get(i)
%timeit for i in range(1000): ex_iTuple.data[i]
%timeit for i in range(1000): ex_list[i]
%timeit for i in range(1000): ex_iTuple[i] # gotcha, probably avoid this :(

83.9 µs ± 7.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
58 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
35.8 µs ± 3.77 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
196 µs ± 3.85 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Elementwise function application with .map() is slightly faster than with a list comprehension:

In [10]:
add_2 = functools.partial(operator.add, 2)

%timeit xtuples.iTuple.range(1000).map(add_2)
%timeit [add_2(x) for x in range(1000)]
%timeit list(map(add_2, range(1000)))

72.2 µs ± 5.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
75.7 µs ± 5.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
62 µs ± 2.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Both fold and cumulative fold are a fair bit faster than the equivalent for loops:

In [11]:
def f_loop_fold():
    acc = 0
    for i in ex_list:
        acc = operator.add(acc, i)
    return acc

%timeit ex_iTuple.fold(operator.add)
%timeit f_loop_fold()

39.6 µs ± 4.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
66.4 µs ± 5.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
def f_loop_cumfold():
    res = []
    acc = 0
    for i in ex_list:
        acc = operator.add(acc, i)
        res.append(acc)
    return res

%timeit ex_iTuple.foldcum(operator.add)
%timeit f_loop_cumfold()

51.7 µs ± 5.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
107 µs ± 4.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Though .filter() is slightly worse than both the for loop and list comprehension implementations:

In [15]:
def f_loop_filter(f):
    res = []
    for i in ex_list:
        if f(i):
            res.append(i)
    return res

f = lambda x: x % 2 == 0

%timeit ex_iTuple.filter(f)
%timeit f_loop_filter(f)
%timeit [v for v in ex_list if f(v)]

119 µs ± 7.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
109 µs ± 7.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
93.8 µs ± 5.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
# append, extend, prepend, pretend, flatten, first, last, slicing (eg. take, tail, etc.)

#### Named Tuple

As nTuple.decorate only adds a couple of methods to the built-in NamedTuple, performance is also very strong.

For instance, NamedTuples are significantly more memory efficient than dicts or vanilla class instances:

In [24]:
@xtuples.nTuple.decorate
class Example(typing.NamedTuple):
    x: int
    s: str

In [25]:
class Example_Cls:
    x: int
    s: str
    
    def __init__(self, x, s):
        self.x = x
        self.s = s
        

In [25]:
# vs dataclass?

In [26]:
      
ex_nTuple = Example(1, "a")
ex_dict = dict(x=1, s="a")
ex_cls = Example_Cls(1, "a")

dict(
    nTuple=asizeof(ex_nTuple),
    dict=asizeof(ex_dict),
    cls=asizeof(ex_cls),
)

{'nTuple': 144, 'dict': 432, 'cls': 352}

Whilst providing comparable field access times:

In [28]:
%timeit ex_nTuple.x
%timeit ex_nTuple[0]
%timeit ex_dict["x"]
%timeit ex_cls.x

34.4 ns ± 6.95 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
33.7 ns ± 5.55 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
39.6 ns ± 4.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
33.2 ns ± 3.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Sorting that does what you'd expect (versus dict, not sortable):

Hashable:

When used with iTuple, also *recursively* hashable:

Also, even more efficient in tuple (because a constant?):

#### Eager evaluation

The one performance caveat is that many methods which are canonically lazy, returning generators, are instead treated eagerly - however this is something I am actively working on.

### xtuples.json

xtuples.json provides base classes for weakly-rich json encoding / decoding (rich in that classes are preserved, weak in that this is based on class name alone and no further checks or guarantees are provided).

## License

`xtuples` is distributed under the terms of the [MIT](https://spdx.org/licenses/MIT.html) license.


#### Auto reload

In [14]:
%load_ext autoreload
%autoreload 2
import importlib
importlib.reload(xtuples)

<module 'src.xtuples' from 'C:\\xtuples\\src\\xtuples\\__init__.py'>